# Hysteresis Processing

## Install and import packages

In [1]:
import pmagpy.rockmag as rmag
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Import local data in MagIC format
In this demonstration we will be using local data.

The data is from the following publication:
- Swanson-Hysell, N. L., Avery, M. S., Zhang, Y., Hodgin, E. B., Sherwood, R. J., Apen, F. E., et al. (2021). The paleogeography of Laurentia in its early years: New constraints from the Paleoproterozoic East-Central Minnesota Batholith. Tectonics, 40, e2021TC006751. https://doi.org/10.1029/2021TC006751

In [2]:
# set the dir_path to the directory where the measurements.txt file is located
dir_path = '../example_data/ECMB'

# set the name of the MagIC file
ipmag.unpack_magic('magic_contribution_20213.txt', 
                     dir_path = dir_path,
                     input_dir_path = dir_path,
                     print_progress=False)

# create a contribution object from the tables in the directory
contribution = cb.Contribution(dir_path)
measurements = contribution.tables['measurements'].df
specimens = contribution.tables['specimens'].df

1  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/contribution.txt
1  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/locations.txt
90  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/sites.txt
312  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/samples.txt
1574  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/specimens.txt
17428  records written to file  /Users/yimingzhang/Github/RockmagPy-notebooks/example_data/ECMB/measurements.txt
-I- Using online data model
-I- Getting method codes from earthref.org
-I- Importing controlled vocabularies from https://earthref.org


## Get hysteresis data using the method code

The method codes relevant to hysteresis loops are:
 - `LP-HYS` for regular hysteresis loops
 - `LP-HYS-O` for hysteresis loops as a function of orientation
 - `LP-HYS-T` for hysteresis loops as a function of temperature

We can filter the data to only have data for `LP-HYS`.

In [3]:
hyst_measurements = measurements[measurements['method_codes'] == 'LP-HYS']

We can see what experimental hysteresis data are available in this dataset:

In [4]:
hyst_experiments = rmag.make_experiment_df(hyst_measurements)
hyst_experiments

,specimen,method_codes,experiment
0,NED1-5c,LP-HYS,IRM-VSM3-LP-HYS-218845
1,NED18-2c,LP-HYS,IRM-VSM3-LP-HYS-218847
2,NED2-8c,LP-HYS,IRM-VSM3-LP-HYS-218849
3,NED4-1c,LP-HYS,IRM-VSM3-LP-HYS-218853
4,NED6-6c,LP-HYS,IRM-VSM3-LP-HYS-218855


## Process hysteresis data for one experiment

We can visualize that data from one of the experiments using the `rmag.process_hyst_loop function`. The details behind the function are detailed in the hysteresis_processing_walk_through.ipynb notebook.

In [5]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-218845'
specimen_name = 'NED1-5c'

experiment_hyst = measurements[measurements['experiment'] == experiment_name].reset_index(drop=True)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, experiment_hyst['magn_mass'].values, specimen_name)
NED1_5c_hyst_process_result = experiment_results

## Process hysteresis data for all experiments

In [6]:
results = rmag.process_hyst_loops(hyst_experiments,measurements)

In [7]:
for i in hyst_experiments.index:
    experiment = hyst_experiments['experiment'][i]
    specimen = hyst_experiments['specimen'][i]
    experiment_hyst = measurements[measurements['experiment'] == experiment].reset_index(drop=True)
    experiment_hyst_result = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, experiment_hyst['magn_mass'].values, specimen)
    

In [8]:
experiment_hyst_result

{'gridded_H': array([ 1.000000000000000e+00,  9.950027500000000e-01,
         9.900055000000001e-01,  9.850082500000001e-01,
         9.800110000000002e-01,  9.750137500000002e-01,
         9.700165000000003e-01,  9.650192500000003e-01,
         9.600220000000004e-01,  9.550247500000004e-01,
         9.500275000000005e-01,  9.450302500000005e-01,
         9.400330000000006e-01,  9.350357500000006e-01,
         9.300385000000007e-01,  9.250412500000007e-01,
         9.200440000000008e-01,  9.150467500000008e-01,
         9.100495000000008e-01,  9.050522500000009e-01,
         9.000550000000009e-01,  8.950577500000010e-01,
         8.900605000000010e-01,  8.850632500000011e-01,
         8.800660000000011e-01,  8.750687500000012e-01,
         8.700715000000012e-01,  8.650742500000013e-01,
         8.600770000000013e-01,  8.550797500000014e-01,
         8.500825000000014e-01,  8.450852500000015e-01,
         8.400880000000015e-01,  8.350907500000015e-01,
         8.300935000000016e-01,  8.

## export hysteresis loop statistics into a MagIC specimens data table

In [9]:
rmag.add_hyst_stats_to_specimens_table(specimens, 'IRM-VSM3-LP-HYS-218845', NED1_5c_hyst_process_result)

In [10]:
specimens

,citations,critical_temp,critical_temp_type,description,dir_comp,dir_dang,dir_dec,dir_inc,dir_mad_free,dir_n_comps,...,rem_hirm_mass,rem_sratio,rem_sratio_back,rem_sratio_forward,result_quality,sample,software_packages,specimen,volume,weight
specimen name,,,,,,,,,,,,,,,,,,,,,
NED1-1b,This study,NaN,None,None,mc,2.8,159.7,62.5,1.0,1.0,...,NaN,NaN,NaN,NaN,g,NED1-1,pmagpy-4.2.25: demag_gui,NED1-1b,NaN,NaN
NED1-1b,This study,NaN,None,None,mc,2.8,139.2,74.1,1.0,1.0,...,NaN,NaN,NaN,NaN,g,NED1-1,pmagpy-4.2.25: demag_gui,NED1-1b,NaN,NaN
NED1-1b,This study,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NED1-1,None,NED1-1b,NaN,NaN
NED1-2b,This study,NaN,None,None,mc,3.4,164.1,66.9,0.4,1.0,...,NaN,NaN,NaN,NaN,g,NED1-2,pmagpy-4.2.25: demag_gui,NED1-2b,NaN,NaN
NED1-2b,This study,NaN,None,None,mc,3.4,149.9,76.3,0.4,1.0,...,NaN,NaN,NaN,NaN,g,NED1-2,pmagpy-4.2.25: demag_gui,NED1-2b,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NED1-5c,This study,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,...,-0.000007,0.931,0.1,1.0,None,NED1-5,None,NED1-5c,NaN,0.000154
NED18-2c,This study,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,...,0.000066,0.938,0.1,1.0,None,NED18-2,None,NED18-2c,NaN,0.000254
NED2-8c,This study,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,...,0.000074,0.846,0.1,1.0,None,NED2-8,None,NED2-8c,NaN,0.000188
